## Unsupervised Capstone Project

### Ten articles with 10 different authors have been selected for this project. The articles are obtained from free Gutenberg Project and they are all Christmas-related.

In [1]:
import timeit
start=timeit.default_timer()

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re

from collections import Counter

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import spacy
nlp = spacy.load('en')
from spacy.lang.en.stop_words import STOP_WORDS

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [6]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

## Load files

In [8]:
filename=['brady.txt','butler.txt','cutting.txt','dawson.txt','dyke.txt',
          'hughes.txt','mitchell.txt','richmond.txt','rinehart.txt','stretton.txt']


In [9]:
def load_text(filename):
    file=open(filename,'r')
    text=file.read()
    file.close
    return text

In [10]:
def clean_text(text):
    text=text.lower()
    text=text.replace("ï»¿","")
    text=text.replace("\n"," ")
    text=text.replace("*"," ")
    text=text.replace("-"," ")
    text=text.replace("\'s"," is")
    text=text.replace("n\'t"," not")
    text=text.replace("\'m"," am")
    text=text.replace("\'ll"," will")
    text=text.replace("in'","ing")
    text=text.replace("mr.","mr")
    text=text.replace("mrs.","mrs")
    text=text.replace("i'd","i would")
    text=text.replace("p. s.","")
    text=text.replace("n. y.","")
    text=text.replace("dec.","dec")
    text=text.replace("1904.","1904")
    text=text.replace("austin.","austin")
    text=text.replace("co.","co")
    text=text.replace("new york.","new york")
    text=text.replace("kringle.","kringle")
    text=text.replace("dear.","dear ")
    text=text.replace("not.","not ")
    text=text.replace("do.","do ")
    text=text.replace("nan.","nan ")
    text=text.replace("yes.","yes ")
    text=text.replace("song.","song ")
    text=text.replace("boy.","boy ")
    text=text.replace("will.","will ")
    text=text.replace("now.","now ")
    text=text.replace("child.","child ")
    text=text.replace("reconciled.","reconciled ")
    
    text=text.replace('"'," ")
    text=text.replace('_'," ")
    text=text.replace('?'," ")
    text=text.replace('!'," ")
    text=text.replace(','," ")
    text=text.replace(":"," ")
    text=text.replace(";"," ")
    text=text.replace("...","")
       
    text=re.sub("[\(\[].*?[\)\]]"," ",text)
 
    return text

## K-Means Clustering

In [11]:
df = pd.DataFrame()
vectorizer = TfidfVectorizer(stop_words='english',max_features=50)
for i in range(len(filename)):
    text=load_text(filename[i])
    text=clean_text(text)
    text_clean=text.split()
    
    X = vectorizer.fit_transform(text_clean)
    X_np=X.todense()
    X_df=pd.DataFrame(X_np)
    
    author=filename[i].replace('.txt','')
    X_df['author']=author   
    
    df=df.append(X_df)

In [12]:
df.shape

(72036, 51)

In [13]:
from sklearn.preprocessing import normalize
x=df.drop(['author'],axis=1)
x=normalize(x)
y=df['author']

In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, 
                                                    y,
                                                    test_size=0.25,
                                                    random_state=0)

In [15]:
k=10
model = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=1)
model.fit(x_train)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=10, n_init=1, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [16]:
y_pred=model.predict(x_test)

In [17]:
from sklearn import metrics
    
metrics.adjusted_rand_score(y_test, y_pred)

0.0006803733682703563

In [18]:
y_pred=model.predict(x_train)
metrics.adjusted_rand_score(y_train, y_pred)

0.0005295745094852884

In [19]:
from sklearn.metrics import pairwise_distances

model.fit(x)
labels=model.labels_

score=metrics.silhouette_score(x, labels, metric='euclidean')
print(score)

0.852437953182771


In [20]:
z

NameError: name 'z' is not defined

## Unsupervised Feature Generation

In [ ]:
text_list=[]
len_list=[None]*(len(filename)+1)
len_list[0]=0
for i in range(0,len(filename)):
    text=load_text(filename[i])
    text=clean_text(text)
    temp_list=text.split(".")
    len_list[i+1]=len_list[i]+len(temp_list)
    text_list=text_list+temp_list

In [ ]:
len(text_list)

In [ ]:
len_list

In [ ]:
df=pd.DataFrame(index=range(len(text_list)),columns=['author'])
for i in range(0,(len(len_list)-1)):
    author=filename[i].replace('.txt','')
    x=len_list[i]
    y=len_list[i+1]
    df['author'][x:y]=author

In [ ]:
len(text_list)

In [ ]:
df.shape

In [ ]:
len_list

In [ ]:
len(text_list)

## Bag of Words

In [ ]:
vectorizer=CountVectorizer(stop_words="english", 
                           analyzer='word', 
                           ngram_range=(1, 4), 
                           max_df=1.0, 
                           min_df=1, 
                           max_features=6000)

In [ ]:
x = vectorizer.fit_transform(text_list)
x = x.toarray()
x.shape

In [ ]:
y=df['author']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, 
                                                    y,
                                                    test_size=0.25,
                                                    )

In [ ]:
rfc = ensemble.RandomForestClassifier()
rfc.fit(x_train, y_train)
print('Training set score:', rfc.score(x_train, y_train))
print('\nTest set score:', rfc.score(x_test, y_test))

In [ ]:
lr = LogisticRegression()
lr.fit(x_train, y_train)
print('Training set score:', lr.score(x_train, y_train))
print('\nTest set score:', lr.score(x_test, y_test))

## TFIDF

In [ ]:
vectorizer = TfidfVectorizer(max_df=0.5, # drop words that occur in more than half the paragraphs
                             min_df=2, # only use words that appear at least twice
                             stop_words='english', 
                             lowercase=True, #convert everything to lower case (since Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS)
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )

In [ ]:
x = vectorizer.fit_transform(text_list)
x = x.toarray()
x.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, 
                                                    y,
                                                    test_size=0.25,
                                                    )

In [ ]:
rfc = ensemble.RandomForestClassifier()
rfc.fit(x_train, y_train)
print('Training set score:', rfc.score(x_train, y_train))
print('\nTest set score:', rfc.score(x_test, y_test))

In [ ]:
lr = LogisticRegression()
lr.fit(x_train, y_train)
print('Training set score:', lr.score(x_train, y_train))
print('\nTest set score:', lr.score(x_test, y_test))